# Axisymmetric hyperboloid with free ends with linear HHJ formulation
In this notebook we consider an axisymmetric hyperboloid given by the equation
$$
x^2+y^2=R^2+z^2,\qquad z\in[-R,R]
$$
with free boundaries and a periodic body force. Due to symmetry we consider only one eight of the geometry and prescribe appropriate symmetry boundary conditions.

The material and geometrical properties are
$$
E = 2.85\times 10^4,\qquad \nu = 0.3,\qquad R=1, \qquad t\in\{1,0.1,0.01,0.001\},
$$
the force is given by $p=t^310^4\cos(2\zeta)\hat{\nu}$, $\zeta\in [0,2\pi)$, and we consider the HHJ formulation for linear Kirchhoff-Love shells:
\begin{align*}
\mathcal{L}(u,\boldsymbol{m})&=\int_S \frac{t}{2}\|\mathcal{I}^{\mathrm{Reg}}\mathrm{sym}(P_S\nabla_S u)\|_{\mathbb{M}}^2  -\frac{6}{t^3}\|\boldsymbol{m}\|^2_{\mathbb{M}^{-1}} + \sum_{T\in\mathcal{T}} \int_T\boldsymbol{m}: \mathcal{H}_{\hat{\nu}}\,dx \\
&\qquad-\int_{\partial T}\boldsymbol{m}_{\hat\mu\hat\mu}(\nabla_S u^T)_{\hat\nu\hat\mu}\,ds - \int_S f\cdot u\,dx
\end{align*}
with the surface Hessian $\mathcal{H}_{\hat{\nu}} = \sum_{i}\nabla^2_Su_i {\hat{\nu}}_i$, unit normal vector ${\hat{\nu}}$, edge-tangential vector $\hat\tau$, and the co-normal (element-normal) vector $\hat\mu=\hat\nu\times \hat\tau$.

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
import netgen.meshing as meshing

# Geometry and material parameters
r = 1
E, nu = 2.85e4, 0.3


def GenerateMesh(nx, ny):
    # Create geometry and mesh
    mapping = lambda x, y, z: (
        x,
        sqrt(1 + x**2) * cos(pi / 2 * y),
        sqrt(1 + x**2) * sin(pi / 2 * y),
    )
    geom = meshing.SurfaceGeometry(mapping)
    return Mesh(geom.GenerateMesh(quads=False, nx=nx, ny=ny))

In [ ]:
def MaterialStress(mat, E, nu):
    return E / (1 - nu**2) * ((1 - nu) * mat + nu * Trace(mat) * Ptau)


def MaterialStressInv(mat, E, nu):
    return (1 + nu) / E * (mat - nu / (nu + 1) * Trace(mat) * Ptau)


# Solve Kirchhoff-Love shell equations with given mesh, thickness, and polynomial order
def Solve(mesh, thickness, order=2):
    # Angle
    zeta = IfPos(y, atan(z / y), pi / 2)
    # -specialcf.normal(3) such that normal vector points away from the origin
    force = 1e4 * thickness**3 * cos(2 * zeta) * (-specialcf.normal(3))

    # Symmetry boundary conditions on left, top, and bottom boundaries
    fesU = VectorH1(
        mesh,
        order=order,
        dirichletx_bbnd="left",
        dirichlety_bbnd="top",
        dirichletz_bbnd="bottom",
    )
    fesM = HDivDivSurface(mesh, order=order - 1, discontinuous=True)
    fesH = NormalFacetSurface(mesh, order=order - 1, dirichlet_bbnd="left|bottom|top")
    fes = fesU * fesM * fesH

    (u, m, hyb), (du, dm, dhyb) = fes.TnT()
    m, hyb, dm, dhyb = m.Trace(), hyb.Trace(), dm.Trace(), dhyb.Trace()

    Regge = HCurlCurl(mesh, order=order - 1, discontinuous=True)

    gf_solution = GridFunction(fes, name="solution")

    nv = specialcf.normal(mesh.dim)
    tv = specialcf.tangential(mesh.dim)
    cnv = Cross(nv, tv)

    Ptau = Id(mesh.dim) - OuterProduct(nv, nv)

    Hn = (u.Operator("hesseboundary").trans * nv).Reshape((3, 3))
    dHn = (du.Operator("hesseboundary").trans * nv).Reshape((3, 3))

    # Bilinear form
    a = BilinearForm(fes, symmetric=True, condense=True)
    # Membrane part
    a += (
        thickness
        * InnerProduct(
            MaterialStress(Interpolate(Sym(Ptau * Grad(u).Trace()), Regge), E, nu),
            Interpolate(Sym(Ptau * Grad(du).Trace()), Regge),
        )
    ).Compile() * ds
    # Bending part
    a += (
        -12 / thickness**3 * InnerProduct(MaterialStressInv(m, E, nu), dm)
    ).Compile() * ds
    a += (InnerProduct(dm, Hn) + InnerProduct(m, dHn)).Compile() * ds
    a += (
        -m[cnv, cnv] * (Grad(du).Trace()[nv, cnv] - dhyb * cnv)
        - dm[cnv, cnv] * (Grad(u).Trace()[nv, cnv] - hyb * cnv)
    ).Compile() * ds(element_boundary=True)

    # Linear form with force
    f = LinearForm(fes)
    f += InnerProduct(force, du).Compile() * ds

    # Assemble and solve
    a.Assemble()
    f.Assemble()
    r = f.vec.CreateVector()
    if a.condense:
        r.data = f.vec
        r.data += a.harmonic_extension_trans * r
        inv = a.mat.Inverse(fes.FreeDofs(True), inverse="sparsecholesky")
        gf_solution.vec.data = solvers.PreconditionedRichardson(
            a, r, inv, maxit=3, tol=1e-13, printing=False
        )
        gf_solution.vec.data += a.harmonic_extension * gf_solution.vec
        gf_solution.vec.data += a.inner_solve * r
    else:
        inv = a.mat.Inverse(fes.FreeDofs(), inverse="umfpack")
        r.data = f.vec
        gf_solution.vec.data = solvers.PreconditionedRichardson(
            a, r, inv, maxit=3, tol=1e-13, printing=False
        )

    return gf_solution

We compute a reference solution by reducing the shell equations to a 1D problem and solve it with a 1D FEM code. From the 1D solution we can recover the full 3D solution to compute the error.

In [ ]:
%run reference_values_hyperboloid.py
from ngsolve.webgui import Draw

In [ ]:
# Point of interest
point_P = (0, 0, r)
# Polynomial order of displacement field
order = 2

# List of grids and thicknesses
grids = [
    (4, 4),
    (7, 7),
    (12, 12),
    (24, 24),
    (48, 48),
    (96, 96),
    # (192, 192),
]
thicknesses = [1, 0.1, 0.01, 0.001]

# Store results
convergence = {t: [] for t in thicknesses}
# Get reference solutions
print("Computing reference solutions")
ref_sol = {
    t: Get3DSolution(n=300, order=3, t=t, ReissnerMindlin=False) for t in thicknesses
}
print("Reference solutions computed")

n = specialcf.normal(3)
Ptau = Id(3) - OuterProduct(n, n)

# Iterate over grid and thicknesses to compute solutions
with TaskManager():
    for grid in grids:
        print("grid = ", grid)
        mesh = GenerateMesh(*grid).Curve(order)
        for t in thicknesses:
            print("thickness = ", t)
            gf_solution = Solve(mesh=mesh, thickness=t, order=order)
            gfu, gfm, _ = gf_solution.components

            # Compute point deflection and relative L2 error of displacement field
            convergence[t].append(
                (
                    gf_solution.space.ndof,
                    abs((gf_solution.components[0](mesh(*point_P, BND)))[2]),
                    sqrt(
                        Integrate(
                            (gf_solution.components[0] - ref_sol[t]) ** 2, mesh, BND
                        )
                    )
                    / sqrt(
                        Integrate(ref_sol[t] ** 2, mesh, BND)
                    ),  # displacement L2 error
                )
            )

Plot the $L^2$ and point deflection errors.

In [ ]:
import matplotlib.pyplot as plt

# Reference values at point of interest
ref_values = {1: 0.8549465, 0.1: 0.1856305, 0.01: 0.1502913, 0.001: 0.1498749}

plt.figure()
plt.title("Relative L2 error")
for t, values in convergence.items():
    dofs, P_r, err_u_l2 = zip(*values)
    plt.loglog(dofs, err_u_l2, "-*", label=f"t={t}")
# Reference line (quadratic convergence)
plt.loglog(
    dofs[1:-1], [1 / dof for dof in dofs[1:-1]], "--", label="$\\mathcal{O}(h^2)$"
)
plt.xlabel("ndof")
plt.ylabel("relative error")
plt.legend()


plt.figure()
plt.title("Point deflection error")
for t, values in convergence.items():
    dofs, P_r, err_u_l2 = zip(*values)
    err = [abs(P - ref_values[t]) / abs(ref_values[t]) for P in P_r]
    plt.loglog(dofs, err, "-*", label=f"t={t}")
# Reference line (quadratic convergence)
plt.loglog(
    dofs[1:-1], [1 / dof for dof in dofs[1:-1]], "--", label="$\\mathcal{O}(h^2)$"
)
plt.xlabel("ndof")
plt.ylabel("relative error")
plt.legend()

plt.show()

In [ ]:
print("l2 error \n")
for t, values in convergence.items():
    print(f"t = {t}")
    dofs, P_r, err_u_l2 = zip(*values)

    for i, (dof, err_l2) in enumerate(zip(dofs, err_u_l2)):
        print(f"({dof}, {err_l2} )")


print("point error \n")
for t, values in convergence.items():
    print(f"t = {t}")
    dofs, P_r, err_u_l2 = zip(*values)
    err = [abs(P - ref_values[t]) / abs(ref_values[t]) for P in P_r]
    for i, (dof, err_p) in enumerate(zip(dofs, err)):
        print(f"({dof}, {err_p} )")